<a href="https://colab.research.google.com/github/narhim/kg_quijote/blob/main/quijote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PREAMBLE

In [1]:
!python -m spacy download es_core_news_sm
!pip install unidecode
!pip install span_marker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 171.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.7

In [2]:
import re
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy
from spacy.lang.es.examples import sentences
import unidecode
from unidecode import unidecode
from pathlib import Path
import re
import span_marker
from span_marker import SpanMarkerModel
import requests
from bs4 import BeautifulSoup
import operator
!pwd

/content


In [3]:
from google.colab import drive #To access drive
drive.mount('/content/drive')


Mounted at /content/drive


#FUNCTIONS



In [4]:
def split_text(texto_lista,lista):
	'''Splits text in list of texts and stores in the second list, which is empty.'''
	for c in texto_lista:
		p = re.split('[.]|[?]|[!]|[\)]',c) #Sentence breaker
		for s in p:
			sen = re.split(",|;|:",s)#Phrase breaker
			for ph in sen:
				ws = re.split(" ",ph)#Word splitter
				for w in ws:
					if re.search("¿|-|¡|\(",w):
						w = re.split("¿|-|¡|\(",w)[-1]
					if re.search("—",w):
						if re.match("—",w):
							try:
								w = re.split("—",w)[-1]
							except:
								continue
						else:
							w = re.split("—",w)[0]
					if re.search("\r",w):
						s = re.split("\r",w)
						lista.append(s[0].lower())
						lista.append(s[1].lower())

					elif len(w) >= 1:
						lista.append(unidecode(w.lower()))
	lista = [w.strip('<*') for w in lista]
	lista = [w.strip('>*') for w in lista]
	lista = [w.strip('\n') for w in lista if len(w)>1]
	lista = [w.strip('\'*') for w in lista]
	lista = [w.strip('»') for w in lista]
	lista = [w.strip('«') for w in lista]
	lista = [w.strip('\"') for w in lista if len(w)>1]
	return lista


In [5]:
def freq_sort(list_ngrams):
	'''Computes frequency of elements in list and returns a sorted dict top to bottom with elements as keys and frequencies as values.'''
	freq_tokens = {}
	for t in list_ngrams:
		if t in freq_tokens:
			freq_tokens[t] += 1
		else:
			freq_tokens[t] = 1
	sorted_freq_tokens = sorted(freq_tokens.items(), key=operator.itemgetter(1))
	sorted_freq_tokens.reverse()
	dict_sort_freq = {}
	for t in sorted_freq_tokens:
		dict_sort_freq[t[0]] = t[1]
	return dict_sort_freq


In [6]:
def write_freq(name,dictionary):
	'''Writes dict into file with given name.'''
	f = open(name,"w")
	for k,v in dictionary.items():
		f.write(str(k) + "\t" + str(v) + "\n")

#GETTING "EL QUIJOTE" AND ANALYZING IT

##Scrapping

In [7]:
url = "https://www.gutenberg.org/cache/epub/2000/pg2000-images.html"
rget = requests.get(url)
soup = BeautifulSoup(rget.text, "lxml")
section_titles_raw = soup.findAll("h2")
section_titles = [t.text for t in section_titles_raw[2:-1]]
chapters_raw = soup.findAll("h3")
chapters = [t.text for t in chapters_raw[6:]]#Capítulo I
paragraphs_raw = soup.findAll("p")
paragraphs = [t.text for t in paragraphs_raw[65:]]#Comienza "En un lugar de la Mancha..."


<ipython-input-7-f214a23909f9>:4: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  section_titles_raw = soup.findAll("h2")
<ipython-input-7-f214a23909f9>:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  chapters_raw = soup.findAll("h3")
<ipython-input-7-f214a23909f9>:8: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  paragraphs_raw = soup.findAll("p")


##Splitting, n-gram, frequency, and writing into tsv.

In [8]:
#1-grams
titles = []
titles = split_text(chapters,titles)
text = []
text = split_text(paragraphs,text)
tokens = titles + text
freq_tokens = freq_sort(tokens)
write_freq("freq_tokens.tsv",freq_tokens)
#2-grams
bigrams = [(t,tokens[n+1]) for n,t in enumerate(tokens[:-1])]#build
freq_bigrams = freq_sort(bigrams)
write_freq("freq_bigrams.tsv",freq_bigrams)
#3-grams
trigrams = [(t,tokens[n+1],tokens[n+2]) for n,t in enumerate(tokens[:-2])]#build
freq_trigrams = freq_sort(trigrams)
write_freq("freq_trigrams.tsv",freq_trigrams)
#4-grams
tetragrams = [(t,tokens[n+1],tokens[n+2],tokens[n+3]) for n,t in enumerate(tokens[:-3])]#build
freq_tetragrams = freq_sort(tetragrams)
write_freq("freq_tetragrams.tsv",freq_tetragrams)
#5-grams
pentagrams = [(t,tokens[n+1],tokens[n+2],tokens[n+3],tokens[n+4]) for n,t in enumerate(tokens[:-4])]#build
freq_pentagrams = freq_sort(pentagrams)
write_freq("freq_pentagrams.tsv",freq_pentagrams)
#Write raw text for NER
f = open("quijote.txt","w")
for p in paragraphs:
	f.write(p + "\n")


#LOAD MODELS

##Wikineural Model: Multilingual

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
model_wiki = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "En un lugar de la Mancha, de cuyo nombre no quie\nro acordarme, no ha mucho tiempo q\nue vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna diferencia en los autores que deste caso escriben; aunque, por conjeturas verosímiles, se deja entender que se llamaba Quejana. Pero esto importa poco a nuestro cuento; basta que en la narración dél no se salga un punto de la verdad."
ner_results = model_wiki(example)
print(ner_results)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'LOC', 'score': 0.95070237, 'word': 'Mancha', 'start': 18, 'end': 24}, {'entity_group': 'PER', 'score': 0.7377888, 'word': 'Quijada', 'start': 932, 'end': 939}, {'entity_group': 'PER', 'score': 0.7487553, 'word': 'Quesada', 'start': 943, 'end': 950}, {'entity_group': 'PER', 'score': 0.49851927, 'word': 'Quejana', 'start': 1094, 'end': 1101}]


##Spacy: Spanish

In [11]:
model_spacy = spacy.load("es_core_news_sm")
r = model_spacy(example)
for token in r:
  if (token.ent_type_):
    print(token.text,token.ent_type_)

la LOC
Mancha LOC
Una MISC
olla MISC
de MISC
algo MISC
más MISC
vaca MISC
El MISC
resto MISC
della MISC
concluían MISC
sayo MISC
de MISC
velarte MISC
Tenía PER
Frisaba LOC
Quieren MISC
decir MISC
Quijada LOC
Quesada LOC
Quejana PER


##Bert Based: Spanish

In [12]:
# Download from the 🤗 Hub
model_bert = SpanMarkerModel.from_pretrained("alvarobartt/bert-base-multilingual-cased-ner-spanish")
entities = model_bert.predict(example)
for e in entities:
  print(e["span"],e["label"])

config.json:   0%|          | 0.00/3.39k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

lugar de la Mancha LOC
Quesada LOC


#CLASSIFY

Read file

In [14]:

pos_file = Path("drive/MyDrive/nn_project_trials/quijote.txt")
assert(pos_file.exists())
#with open(pos_file) as f:
#  l = f.readlines()
#  r = nlp(l)
#  print(len(r)) #Safety check: whole text is classified

In [15]:
results = []
with open(pos_file) as f:
  for l in f:
    r = model_wiki(l)
    results.append(r)

30877


In [17]:

ent_rel_dict_spacy = {}
#doc = nlp(sentences[0])
#print(doc.text)
#for token in doc:
#    print(token.text, token.pos_, token.dep_,token.ent_type_)
results_spacy = []
with open(pos_file) as f:
  for l in f:
    r = model_spacy(l)
    results_spacy.append(r)
    #for token in r:
    #    if (token.ent_type_) and (token.text not in ent_rel_dict_spacy):
    #        #print(token.text,token.ent_type_,token.lemma_)
    #        ent_rel_dict_spacy[token.text.lower()] = token.ent_type_
print(len(results_spacy))


30877


In [ ]:

# Run inference
ent_rel_dict_bert = {}
results_bert = []
with open(pos_file) as f:
  for l in f:
    entities = model_bert.predict(l)
    results_bert.append(entities)
    #try:
    #  container = entities[0]
    #  entity = container["span"].lower()
    #  if entity not in ent_rel_dict_bert:
    #    ent_rel_dict_bert[container["span"].lower()] = container["label"]
    #except:
    #  continue
print(len(results_bert))

Write results into file

In [ ]:
f = open("drive/MyDrive/nn_project_trials/ner_quijote.txt","w")
for k in r:
  if len(k)>0:
    f.write(str(k[0]["entity_group"]) + "\t" + str(k[0]["word"]) + "\n")

To avoid reclassifying every time it disconnects, read results from file and apply first filter

In [ ]:
#Read file with entities, build into dict with entity as key and filter out
ent_rel_dict_wiki = {}
with open("drive/MyDrive/nn_project_trials/ner_quijote.txt","r") as f:
	for line in f:
		#Separate relation from entity
		elements = re.split("\t",line)
		entity = elements[1].strip("\n").lower()
		relation = elements[0]
		#Filter out len<4. Filters out "Cid"
		if entity not in ent_rel_dict_wiki:
			ent_rel_dict_wiki[entity] = relation


In [ ]:
write_dict_file(ent_rel_dict_wiki,"drive/MyDrive/nn_project_trials/wiki_entities.tsv")
write_dict_file(ent_rel_dict_spacy,"drive/MyDrive/nn_project_trials/spacy_entities.tsv")
write_dict_file(ent_rel_dict_bert,"drive/MyDrive/nn_project_trials/bert_entities.tsv")

Deal with mwe

In [ ]:
filter_set = set(["de","del","iglesia","santa"])
bigrams = []
multigrams = []
unigrams = []
for entity in ent_rel_dict.keys():
  if re.search(" ",entity):
    tokens = re.split(" ",entity)
    if (len(tokens) ==2):
      bigrams.append(tokens)
    else:
      if (filter_set.intersection(set(tokens))) and (len(tokens)<8):
        multigrams.append(tokens)
  else:
    unigrams.append(str(entity))
print(len(bigrams))
filter_set = set(["de","del","el","la","un","una","mi","me"])
filter_bigrams = []
for e in bigrams:
  if len(filter_set.intersection(set(e))) == 0:
      filter_bigrams.append(e)
      #print(e)
  #else:
  #  print(e)
mwe = filter_bigrams + multigrams
print(unigrams)

588
['mancha', 'viernes', 'entresemana', 'podadera', 'complex', 'quejana', 'sobre', 'ocioso', 'tant', 'caza', 'caballerías', 'prosa', 'entender', 'aristóteles', 'belianís', 'esto', 'sigüenza', 'galaor', 'letu', 'libros', 'caballero', 'roncesvalles', 'hércules', 'morgan', 'reinaldos', 'montalbán', 'mahoma', 'galalón', 'república', 'género', 'trap', 'prie', 'bisa', 'golpe', 'semana', 'gonela', 'famoso', 'quijote', 'quijada', 'amadís', 'hepila', 'limpias', 'anda', 'caraculiambro', 'malindrania', 'dulcinea', 'toboso', 'tuer', 'deudas', 'intención', 'rocinante', 'lanza', 'content', 'caballería', 'novel', 'pensamiento', 'armiño', 'apolo', 'manche', 'montiel', 'dichos', 'quien', 'habedes', 'plégaos', 'autor', 'rocí', 'muertos', 'diose', 'sevilla', 'castillos', 'rocinan', 'dama', 'porque', 'máquina', 'abundancia', 'castilla', 'sanlúcar', 'estudiant', 'desayunado', 'miró', 'ñudos', 'donc', 'princesa', 'lanzarote', 'palabra', 'trucha', 'trip', 'negro', 'comer', 'cast', 'venteril', 'mundo', 'seme